In [8]:
import os
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Train a Custom Face Detection Model 
Let's train your own face detector, we can use a CNN-based model. Here i will use PyTorch:

In [9]:
def preprocess_images(image_dir, target_size=(224, 224)):
    images = []
    labels = []  # This will be used if you're working with labeled data like LFW
    # image_folder = os.path.join(image_dir, 'lfw-deepfunneled')  # Path to the folder containing images
    
    for root, dirs, files in os.walk(image_dir):  # Traverse through subfolders
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):  # Ensure you're processing image files
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)
                if img is not None:
                    # Resize image to target size
                    img = cv2.resize(img, target_size)
                    # Convert image to grayscale (optional)
                    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    images.append(img)
                   
    images = np.array(images)
    return images

In [10]:
wf_data_path = "C:/Users/galin/Downloads/LFW dataset/lfw-deepfunneled"
lfw_images = preprocess_images(wf_data_path)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define a basic CNN architecture
class FaceDetectorCNN(nn.Module):
    def __init__(self):
        super(FaceDetectorCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64*54*54, 128)
        self.fc2 = nn.Linear(128, 2)  # Binary classification: face or no face

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Prepare your data
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])

# Use your preprocessed datasets (wider_images/lfw_images) here
train_dataset = datasets.ImageFolder(lfw_images, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
model = FaceDetectorCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):  # Train for 10 epochs
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
